In [1]:
import requests
import time
import pandas as pd
import folium
from folium.plugins import MarkerCluster

API_KEY = 'nuh uh'  
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
SEARCH_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
postal_codes = ['V6V', 'V6W', 'V6X', 'V6Y', 'V7A', 'V7C', 'V7E']
all_results = []

for postal in postal_codes:
    print(f"Searching: all restaurants in {postal}...")
    offset = 0
    while offset <= 190:  # limit + offset <= 240
        params = {
            'location': f'{postal}, Richmond, BC',
            'categories': 'restaurants',  # general restaurant category
            'limit': 50,
            'offset': offset
        }

        response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        businesses = response.json().get('businesses', [])
        if not businesses:
            break

        for biz in businesses:
            all_results.append({
                'id': biz['id'],
                'name': biz['name'],
                'address': ", ".join(biz['location']['display_address']),
                'latitude': biz['coordinates']['latitude'],
                'longitude': biz['coordinates']['longitude'],
                'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])])
            })

        offset += 50
        time.sleep(0.5)

# Convert to DataFrame and remove duplicates
df = pd.DataFrame(all_results)
df.drop_duplicates(subset='id', inplace=True)
df.reset_index(drop=True, inplace=True)
df

Searching: all restaurants in V6V...
Searching: all restaurants in V6W...
Searching: all restaurants in V6X...
Searching: all restaurants in V6Y...
Searching: all restaurants in V7A...
Searching: all restaurants in V7C...
Searching: all restaurants in V7E...


,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,Hr5sm6NpE6K-98PrqHksbQ,Anar Persian Cuisine,"12051 Third Avenue, Richmond, BC V7E 3A2, Canada",49.126127,-123.185905,"Persian/Iranian, Middle Eastern"
3,3H63O0o-PXy7yW3MtJznOQ,The Story Cafe,"2800 Sexsmith Road, Richmond, BC V6X 1V5, Canada",49.193593,-123.127918,"Bars, Pizza, Italian"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
470,CHYGStOcv5MgdIwBpSDgSg,John's Pan Pizza,"12420 No 1 Road, Suite 130, Richmond, BC V7E 6...",49.124770,-123.180870,"Pizza, Salad, Italian"
471,p4bEiFv80KGqdpHztWaD_Q,Good Co Bar - Steveston,"4020 Bayview Street, Steveston, BC V7E 0B3, Ca...",49.123578,-123.180234,"Bars, Chicken Wings"
472,iONl1L8eLbLmZFENHae41Q,Shady Island Seafood Bar & Grill,"3800 Bayview Street, Richmond, BC V7E 6K7, Canada",49.123923,-123.183891,"Seafood, Bars"
473,vKIpb6S6C3Cc4GvDorBn2w,Moncton Pizzeria,"3680 Moncton Street, Richmond, BC V7E 3A4, Canada",49.125025,-123.183609,Pizza


In [2]:
# --- Manually append Domino's and Torake ---
params_list = [
    {'term': "Domino's Pizza", 'location': 'Richmond, BC'},
    {'term': "Torake", 'location': 'Richmond, BC'}
]

for params in params_list:
    params['limit'] = 1
    response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
    biz = response.json().get('businesses', [None])[0]
    if not biz or biz['id'] in df['id'].values:
        continue

    coords = biz['coordinates']
    df.loc[len(df)] = {
        'id': biz['id'],
        'name': biz['name'],
        'latitude': coords['latitude'],
        'longitude': coords['longitude'],
        'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])]),
        'address': ", ".join(biz['location']['display_address']),
    }

df.reset_index(drop=True, inplace=True)
print(f"Final total restaurants: {len(df)}")
df

Final total restaurants: 477


,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,Hr5sm6NpE6K-98PrqHksbQ,Anar Persian Cuisine,"12051 Third Avenue, Richmond, BC V7E 3A2, Canada",49.126127,-123.185905,"Persian/Iranian, Middle Eastern"
3,3H63O0o-PXy7yW3MtJznOQ,The Story Cafe,"2800 Sexsmith Road, Richmond, BC V6X 1V5, Canada",49.193593,-123.127918,"Bars, Pizza, Italian"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
472,iONl1L8eLbLmZFENHae41Q,Shady Island Seafood Bar & Grill,"3800 Bayview Street, Richmond, BC V7E 6K7, Canada",49.123923,-123.183891,"Seafood, Bars"
473,vKIpb6S6C3Cc4GvDorBn2w,Moncton Pizzeria,"3680 Moncton Street, Richmond, BC V7E 3A4, Canada",49.125025,-123.183609,Pizza
474,W3fHTFr9TE6OwP2AOTRdhA,Kuroki Kaiseki Omakase Kitchen,"110-3880 Chatham Street, Richmond, BC V7E 2Z5,...",49.126214,-123.182128,"Japanese, Sushi Bars"
475,v6GRZxd-96gojJrXKFcLdw,Domino's Pizza,"9471 No. 2 Road, Unit 120, Richmond, BC V7E 2C...",49.144227,-123.159686,"Pizza, Chicken Wings, Sandwiches"


In [3]:
# --- Create Folium Map ---
m = folium.Map(location=[49.1666, -123.1336], zoom_start=13, tiles='CartoDB dark_matter')

# Add all restaurants
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1.5,
        color='#00ffff',
        fill=True,
        fill_opacity=1,
        stroke=False
    ).add_to(m)

# Save the map
m.save("richmond_restaurants_map_postal.html")
print("🌐 Map saved as richmond_restaurants_map_postal.html")

🌐 Map saved as richmond_restaurants_map_postal.html
